In [1]:
from snorkel_process_NIH import build_raw_data, snorkel_process,loop_labing
import os 
from bertrnn_preprocess_NIH import bertrnn_process
import pandas as pd
import numpy as np

In [2]:
import torch
with torch.no_grad():
    torch.cuda.empty_cache()

In [3]:
allfile, allsent=build_raw_data('/home/text_download/inhibitor_NIH_ab.csv')

100%|██████████| 3269/3269 [00:01<00:00, 1717.76it/s]


In [4]:
allsent['newpid']=range(len(allsent))

In [5]:
with open('/home/text_download/keylist_NIH_task1.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        keylist=line.split(',')
    
    

In [6]:
valuelist = []
with open('/home/text_download/valuelist_NIH_task1.txt', "r") as f:
    alist =f.read().splitlines()
    #alist =[x.lower() for x in alist]
    for line in alist:
        valuelist.append(line.split(','))

In [7]:

with open('/home/text_download/viruslist.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        viruslist=line.split(',')
        #viruslist =[x.lower() for x in viruslist]

In [8]:
allweaklabf=loop_labing(keylist,valuelist,viruslist)

In [9]:
trainsent,trainlabel,valsent,vallabel,keylist,report=snorkel_process (keylist,allsent,allweaklabf)

100%|██████████| 25739/25739 [00:35<00:00, 723.29it/s]


                               j Polarity  Coverage  Overlaps  Conflicts
keyword_antihivsu              0      [0]  0.000272  0.000272   0.000000
abstract_antihivsu1            1      [0]  0.001476  0.000505   0.000233
keyword_antihcvsu              2      [1]  0.000117  0.000117   0.000000
abstract_antihcvsu1            3      [1]  0.000311  0.000117   0.000000
keyword_anti-parasitesu        4       []  0.000000  0.000000   0.000000
abstract_anti-parasitesu1      5      [2]  0.000466  0.000039   0.000039
keyword_immue_suppresivesu     6       []  0.000000  0.000000   0.000000
abstract_immue_suppresivesu1   7      [3]  0.000427  0.000155   0.000155
keyword_antivirussu            8      [4]  0.000350  0.000350   0.000000
abstract_antivirussu1          9      [4]  0.003924  0.000544   0.000194
keyword_interleukinsu         10      [5]  0.000155  0.000155   0.000000
abstract_interleukinsu1       11      [5]  0.007693  0.000505   0.000350
keyword_cancerdrugsu          12       []  0.000000

In [10]:
startw=np.count_nonzero(trainlabel== 1, axis=0)
defualt=np.max(startw)
squarer = lambda t: defualt/t
trainweight=np.array([squarer(xi) for xi in startw])

In [11]:
testsent=allsent.sent.values
testlabel=allsent.newpid.values


In [12]:
args={"modelname1":'bert-base-uncased',
      "modelname2":'scibert_scivocab_uncased',
      "num_labels":len(trainlabel[0]),
      "hidden_size":256,
      "num_layers":1,
      "bidirectional":1,
      "dropout":0.2,
      "batch_size":10,
      "epochs":100,
      "lr":0.001,
      "seed": 1,
      "early_stop_times":5,
      "science":True
}

In [13]:
norloder,sciloder=bertrnn_process(args,trainsent,valsent,testsent,trainlabel,vallabel,testlabel)

100%|██████████| 25739/25739 [00:03<00:00, 7950.73it/s]


In [14]:
import torch.nn as nn
import torch
from transformers import BertModel
from torch.optim import Adam
from torch.autograd import Variable
from sklearn.metrics import label_ranking_loss
from nltk.tokenize import sent_tokenize,word_tokenize
from transformers import BertModel, BertConfig, AdamW
from tqdm import tqdm

class bert_rnn(nn.Module):
    def __init__(self, args):
        super(bert_rnn, self).__init__()
        self.args = args
        self.emb1=BertModel.from_pretrained(self.args['modelname1'],num_labels = self.args['num_labels'],output_attentions = False,output_hidden_states = False)#.cuda(3)
        self.emb1_size=self.emb1.config.hidden_size
        if self.args['science']==True:
            self.emb2=BertModel.from_pretrained(self.args['modelname2'],num_labels = self.args['num_labels'],output_attentions = False,output_hidden_states = False)#.cuda(3)
            self.emb2_size=self.emb2.config.hidden_size
            self.emb_size=self.emb1_size+self.emb2_size
        else:
            self.emb_size=self.emb1_size
        self.lin1 = nn.Linear(self.emb_size, self.args['hidden_size'])
        if self.args['bidirectional']>1:
            bidirectional=2
            dif=True
        else:
            bidirectional=1
            dif=False
        self.rnn=nn.LSTM(input_size=self.args['hidden_size'], hidden_size =self.args['hidden_size'], num_layers =self.args['num_layers'],batch_first= True,dropout=self.args['dropout'],bidirectional=dif)
        self.lin2 = nn.Linear(self.args['hidden_size'], self.args['num_labels'])
        self.sigmoid=nn.Sigmoid()
    def forward(self,data1,mask1,target,data2=None,mask2=None):
        if self.args['science']==True:
            emb1=self.emb1(data1,attention_mask=mask1)
            last_hidden_states1 = emb1[0]
            emb2=self.emb2(data2,attention_mask=mask2)
            last_hidden_states2 = emb2[0]
            last_hidden_states = torch.cat((last_hidden_states1, last_hidden_states2), 2)
            last_hidden_states = self.lin1(last_hidden_states)
        else:
            emb1=self.emb1(data1)
            last_hidden_states1 = emb1[0]
            last_hidden_states = self.lin1(last_hidden_states1)
        output, _ = self.rnn(last_hidden_states)#,# (self.h0, self.c0))
        output=torch.mean(output, 1)
        out = self.lin2(output)
        out=self.sigmoid(out)
        return out,target
    def parameters (self):
        if self.args['science']==True:
            params=list(self.emb1.parameters())+list(self.emb2.parameters())+list(self.rnn.parameters())+list(self.lin1.parameters())+list(self.lin2.parameters())
        else:
            params=list(self.emb1.parameters())+list(self.rnn.parameters())+list(self.lin1.parameters())+list(self.lin2.parameters())
        return params

In [15]:
def train_(model,loss,optimizer,dataloaders1,epoch,dataloaders2=None):
    allloss=[]
    allbatch=[]
    print('Train')
    if args['science']==True:
        for batch_idx, batch in tqdm(enumerate(zip(dataloaders1, dataloaders2))):
            data1, mask1, target1 = batch[0]
            data2, mask2, target2 = batch[1]
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            target2 = Variable(target2).cuda()
            data2 = Variable(data2).cuda()
            mask2=Variable(mask2).cuda()
            optimizer.zero_grad()
            out,target = model.forward(data1,mask1,target1,data2,mask2)
            lossall = loss(out,target.float())            
            lossall = torch.sum(lossall)
            lossall.backward()
            optimizer.step()
            loss1=lossall.item()
            allloss.append(loss1)
            allbatch.append(batch_idx*epoch)
    else:
        for batch_idx, batch in tqdm(enumerate(dataloaders1)):
            data1, mask1, target1 = batch
            target1 = Variable(target1).cuda(3)
            data1 = Variable(data1).cuda(3)
            mask1=Variable(mask1).cuda(3)
            optimizer.zero_grad()
            out,target = model.forward(data1,mask1,target1)
            lossall = loss(out,target.float())            
            lossall = torch.sum(lossall)
            lossall.backward()
            optimizer.step()
            allloss.append(lossall)
            allbatch.append(batch_idx*epoch)
    return allloss, allbatch
def val_(model,dataloaders1,dataloaders2=None):
    allout=[]
    alltarget=[]
    print('Validation')
    if args['science']==True:
        for batch_idx, batch in tqdm(enumerate(zip(dataloaders1, dataloaders2))):
            data1, mask1, target1 = batch[0]
            data2, mask2, target2 = batch[1]
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            target2 = Variable(target2).cuda()
            data2 = Variable(data2).cuda()
            mask2=Variable(mask2).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1,data2,mask2)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(list(out))
            alltarget.append(list(target))
    else:
        for batch_idx, batch in tqdm(enumerate(dataloaders1)):
            data1, mask1, target1 = batch
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(list(out))
            alltarget.append(list(target))
    return allout,alltarget
def test_(model,dataloaders1,dataloaders2=None):
    allout=[]
    alltarget=[]
    print('test')
    if args['science']==True:
        for batch_idx, batch in tqdm(enumerate(zip(dataloaders1, dataloaders2))):
            data1, mask1, target1 = batch[0]
            data2, mask2, target2 = batch[1]
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            target2 = Variable(target2).cuda()
            data2 = Variable(data2).cuda()
            mask2=Variable(mask2).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1,data2,mask2)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(out)
            alltarget.append(target)
    else:
        for batch_idx, batch in tqdm(enumerate(dataloaders1)):
            data1, mask1, target1 = batch
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(out)
            alltarget.append(target)
    return allout,alltarget
def result(result):
    alllist=[]
    for i in result:
        for j in i:
            alllist.append(j)
    return alllist


In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
torch.cuda.set_device(1)
model= bert_rnn(args)
if torch.cuda.device_count() > 1:
    model=nn.DataParallel(model, device_ids=[1,2,3,4,7])
model.to(device)
params=model.parameters()
optimizer = AdamW(params,lr = 2e-5, eps = 1e-8 )
trainweight=torch.tensor(trainweight).float().cuda()
loss = nn.BCEWithLogitsLoss(pos_weight=trainweight)
alloss=[]
allbatch=[]
dev_lrl=1
vallrl=[]
testpred=[]
testpid=[]
current_early_stop_times=0
for epoch in range(1, args['epochs'] + 1):
    epochloss, epochbatch=train_(model,loss,optimizer,norloder[0],epoch,dataloaders2=sciloder[0])
    alloss.append(epochloss)
    allbatch.append(epochbatch)
    allout,alltarget=val_(model,norloder[1],sciloder[1])
    allout1=result(allout)
    alltarget1=result(alltarget)
    epochlrl=label_ranking_loss(alltarget1,allout1)
    vallrl.append(epochlrl)
    if epochlrl < dev_lrl:
        print("- new best lrl{}".format(epochlrl))
        allout,alltarget=test_(model,norloder[2],sciloder[2])
        allout=result(allout)
        alltarget=result(alltarget)
        allpd=pd.DataFrame(allout,columns=keylist)
        allpd['newpid']=alltarget
        allpd.to_csv('Bert_NIH_task4_Q1_LSTM.csv')
        dev_lrl = epochlrl
        current_early_stop_times = 0
    else:
        current_early_stop_times += 1
        print(current_early_stop_times)
    if current_early_stop_times >= args['early_stop_times'] :
        break;
print ("- early stopping {} epochs without improvement".format(epoch))

0it [00:00, ?it/s]

Train


66it [01:21,  1.24s/it]
0it [00:00, ?it/s]

Validation


17it [00:08,  2.08it/s]
0it [00:00, ?it/s]

...

17it [00:08,  2.07it/s]
0it [00:00, ?it/s]

- new best lrl0.11168831168831168
test


2574it [20:40,  2.08it/s]
0it [00:00, ?it/s]

Train


5it [00:04,  1.11it/s]

In [ ]:
allpd=pd.DataFrame(allout,columns=keylist)
allpd['newpid']=alltarget
allpd.to_csv('Bert_NIH_Task1_LSTM.csv')

In [ ]:
len(next(iter(sciloder[2]))[0][1])

In [ ]:
result(alltarget)